# Intersection

#### Welcome to this final exercise in Collinearity!

</style> 

<div class="alert alert-info"> <strong>NAME: </strong
    
{ write your name.surname between the brackets (like that name.surname }
 
</div>

We now have a pretty good idea of what the **Collinearity equations** can help us achieve. 

We can firstly solve for the parameters of a single camera (location and orientation: $X$, $Y$, $Z$ and omega, phi and kappa) given 3 control points and secondly; the equations help in setting the foundation to iteratively refine a Bundle Adjustment (solving for many images simultaneously) _---both of these are through resection._ 

We now move onto the exciting bit. We want to create 3D content.  
<img style="float:right;" src="./img/stereo.png" width= 40% /> 

We do so through measuring the same feature in two of more **_correctly oriented_** photographs by intersecting thier image rays. Because we include the location and orientation of the cameras we use a **_least squares_** method.  

Least Squares is the most effective technique to solve such challenges as it reduces the error between observed and estimated values. Least squares also handles uncertainty. It considers errors in the estimation process, weighting observations based on their reliability. Lastly; least squares can handle overdetermined systems _---where there are more observations than unknowns---_ by finding the best-fit solution that satisfies all observations as closely as possible. Consider **_every pixel_**, in an image, is solved with two collinearity equations. But to recover the 3D object we have to measure two images. **_So everything twice_**. Four equations with three unknowns **_and_** the six exterior orientation parameters (treated as observables with uncertainty) from the bundle adjustment. Now consider the same feature in multiple (three, four or five) images.

We should however take care. Adding more observations does not automatically improve the solution. While redundancy does improve precision and enhance our ability to detect blunders; any bias (typically camera errors or not capturing the entire scene) will dominate the result.

<div class="alert alert-danger">
  <strong>REQUIRED!</strong> 
  
You are required to insert your outputs and any comment into this document. The document you submit should therefore contain the existing text in addition to:

 - Plots and other outputs from executing the code chunks
 - Discussion of your plots and other outputs as well as conclusions reached.
 - This should also include any hypotheses and assumptions made as well as factors that may affect your conclusions.
</div>

In [1]:
#- load the magic
from sympy import sin, cos, Matrix, symbols, lambdify
import numpy as np
import pandas as pd

<div class="alert alert-block alert-success"> <strong> For this exercise we will perform an intersection 
</strong> with a least squares adjustment. 

We will feed image coordinates of the same feature, from two or more images, along with the exterior orientation parameters of thier cameras into a least squares adjustment and iterate until they convergence.  
<br>
A secondary focus is an opportunity to become familiar with the Python programming language and to experiment with changing functions, variables and parameters and to understand how these affect the result.
</div>

In [2]:
input = "./data/inputRes01.txt"

In [3]:
#- look

# For I.O.
with open(input) as fin:
    f = float(fin.readline())   # The focal length in mm
# xp yp XL YL ZL O P K SigXL SigYL SigZL SigO SigP SigK
data = pd.read_csv(
    input,
    delimiter=' ',
    usecols=range(1, 15),
    #names=[str(i) for i in range(14)],
    skiprows=1, names=['xp', 'yp', 'XL', 'YL', 'ZL', 'O', 'P', 'K', 'SigXL', 'SigYL', 'SigZL', 'SigO', 'SigP', 'SigK'])

print('focal length:', f)
data

focal length: 15.0


,xp,yp,XL,YL,ZL,O,P,K,SigXL,SigYL,SigZL,SigO,SigP,SigK
0,-3.8706,4.6560,2.000000,1.000000,2.000000,0.00000,0.000000,0.000000,0.000006,0.000009,0.000002,0.000232,0.000146,0.000042
1,2.6926,-4.3098,1.481621,1.889606,2.183236,5.55765,13.912112,15.269225,0.000007,0.000014,0.000004,0.000353,0.000167,0.000057


<div class="alert alert-block alert-warning"><b>QUESTION! </b>  </div>

- **Why is an intersection important? What does a space intersection help us achieve?**

{ click in this cell and write your answer here }

In [4]:
#- https://github.com/otakusaikou/SpaceIntersection and https://github.com/Abbsalehi/Photogrammetry-Space-Intersection

np.set_printoptions(suppress=True)  # Disable scientific notation for numpy


def getInit(xa, ya, EO, f):
    """Compute initial values of unknown parameters"""
    xa1, xa2 = xa.ravel()
    ya1, ya2 = ya.ravel()

    X1, Y1, Z1 = EO[0, :]
    X2, Y2, Z2 = EO[1, :]

    B = np.sqrt((X2 - X1)**2 + (Y2 - Y1)**2)    # The baseline
    pa = ya1 - ya2                              # The parallax

    H = (Z1 + Z2) / 2

    # Compute arbitrary horizontal coordinates with formula 8-5~8-7
    XA = B * (xa1 / pa)
    YA = B * (ya1 / pa)
    ZA = H - (B * f) / pa

    # Compute the transformation parameters between
    # arbitrary and true object coordinate system
    a = np.cos(np.arctan2((X2 - X1), (Y2 - Y1)))
    b = np.sin(np.arctan2((X2 - X1), (Y2 - Y1)))
    Tx = X1
    Ty = Y1

    # Transform the horizontal coordinates of arbitrary object point
    # and use the result as initial values
    XA2 = a * XA - b * YA + Tx
    YA2 = a * YA + b * XA + Ty

    return XA2, YA2, ZA


def getM(Omega, Phi, Kappa):
    """Compute rotation matrix M"""
    M = np.matrix([
        [
            cos(Phi)*cos(Kappa),
            sin(Omega)*sin(Phi)*cos(Kappa) + cos(Omega)*sin(Kappa),
            -cos(Omega)*sin(Phi)*cos(Kappa) + sin(Omega)*sin(Kappa)],
        [
            -cos(Phi)*sin(Kappa),
            -sin(Omega)*sin(Phi)*sin(Kappa) + cos(Omega)*cos(Kappa),
            cos(Omega)*sin(Phi)*sin(Kappa) + sin(Omega)*cos(Kappa)],
        [
            sin(Phi),
            -sin(Omega)*cos(Phi),
            cos(Omega)*cos(Phi)]
        ])

    return M


def getEqn(IO, EO, PT, pt):
    """List observation equations"""
    f, xo, yo = IO
    XL, YL, ZL, Omega, Phi, Kappa = EO
    XA, YA, ZA = PT
    xa, ya = pt

    M = getM(Omega, Phi, Kappa)

    r = M[0, 0] * (XA - XL) + M[0, 1] * (YA - YL) + M[0, 2] * (ZA - ZL)
    s = M[1, 0] * (XA - XL) + M[1, 1] * (YA - YL) + M[1, 2] * (ZA - ZL)
    q = M[2, 0] * (XA - XL) + M[2, 1] * (YA - YL) + M[2, 2] * (ZA - ZL)

    F = Matrix([xa - xo + f * (r / q), ya - yo + f * (s / q)])
    return F


def spaceIntersection(inputFile, s):
    """Perform a space intersection"""
    # For I.O.
    with open(inputFile) as fin:
        f = float(fin.readline())   # The focal length in mm

    # For E.O.
    # xp yp XL YL ZL O P K SigXL SigYL SigZL SigO SigP SigK
    data = pd.read_csv(
        inputFile,
        delimiter=' ',
        usecols=range(1, 15),
        names=[str(i) for i in range(14)],
        skiprows=1)

    EO, SigEO = np.hsplit(data.values[:, 2:], 2)

    # Convert from degrees to radians
    EO[:, 3:] = np.radians(EO[:, 3:])
    SigEO[:, 3:] = np.radians(SigEO[:, 3:])

    # For image points
    xa, ya = np.hsplit(data.values[:, :2], 2)

    # Compute initial values
    X0 = np.matrix(getInit(xa[:2], ya[:2], EO[:2, :3], f)).T

    #print("Initial Values:\n Param\tValue")
    #print("   XA\t%.6f" % X0[0, 0])
    #print("   YA\t%.6f" % X0[1, 0])
    #print("   ZA\t%.6f" % X0[2, 0])
    # print

    # Define variable for inerior orientation parameters
    IO = f, 0, 0

    # Define symbols
    EOs = symbols("XL YL ZL Omega Phi Kappa")   # E.O. parameters
    PTs = symbols("XA YA ZA")                   # Object point coordinates
    pts = symbols("xa ya")                      # Image coordinates

    # Define weight matrix
    err = SigEO.ravel()     # Error vector
    W = np.matrix(np.diag(s**2 / err**2))
    Q = W.I

    # List and linearize observation equations
    F = getEqn(IO, EOs, PTs, pts)
    JFx = F.jacobian(PTs)
    JFl = F.jacobian(EOs)       # Jacobian matrix for observables

    # Create lambda function objects
    FuncJFx = lambdify((EOs+PTs), JFx, 'numpy')
    FuncJFl = lambdify((EOs+PTs), JFl, 'numpy')
    FuncF = lambdify((EOs+PTs+pts), F, 'numpy')

    numPt = len(data)

    # Create observable array as argument of function objects
    l = np.zeros((numPt, 11))
    l[:, :6] = EO
    l[:, 6:9] = X0[:, :].T
    l[:, 9] += xa.ravel()
    l[:, 10] += ya.ravel()

    dX = np.ones(1)                              # Initial value for iteration

    # Iteration process
    lc = 0          # Loop count
    dRes = 1.       # Termination criteria
    res = 1.        # Initial value of residual
    while dRes > 10**-12 and lc < 20:
        # Compute coefficient matrix and constants matrix
        A = np.zeros((2 * numPt, len(err)))
        B = np.zeros((2 * numPt, 3))

        Ai = FuncJFl(*np.hsplit(l, 11)[:-2])
        Bi = FuncJFx(*np.hsplit(l, 11)[:-2])
        F0 = np.matrix(-FuncF(*np.hsplit(l, 11)).T.reshape(-1, 1))

        for i in range(numPt):
            A[2*i:2*(i+1), 6*i:6*(i+1)] = Ai[:, :, i].reshape(2, 6)
            B[2*i:2*(i+1), :] = Bi[:, :, i].reshape(2, 3)

        A = np.matrix(A)
        B = np.matrix(B)

        # Solve the unknown parameters
        AT = A.T.copy()
        Qe = (A * Q * AT)
        We = Qe.I
        N = (B.T * We * B)                  # Compute normal matrix
        t = (B.T * We * F0)                 # Compute t matrix
        dX = N.I * t                        # Compute unknown parameters
        V = Q * AT * We * (F0 - B * dX)     # Compute residual vector

        X0 += dX            # Update initial values
        l[:, 6:9] += dX[:, :].T

        # Update termination criteria
        if lc > 1:
            dRes = abs(((V.T * W * V)[0, 0]/res) - 1)
        res = (V.T * W * V)[0, 0]

        # Compute sigma0
        s0 = (res / (B.shape[0] - B.shape[1]))**0.5

        lc += 1

    # Compute other informations
    SigmaXX = s0**2 * N.I
    paramStd = np.sqrt(np.diag(SigmaXX))
    XA, YA, ZA = np.array(X0).flatten()
    
    # Print outputs/Ground Coordinate 
    print ("Object point coordinates:")
    #print(XA)
    #print(YA)
    #print(ZA)
    
    return XA, YA, ZA

s = 0.005

In [5]:
#- execute
spaceIntersection(input, s)

Object point coordinates:


(1.4839212533482895, 1.6207994442351308, 3.453874492588568e-06)

<div class="alert alert-block alert-warning"><b>QUESTION! </b>  </div>

- **What are the `Initial Values` that initiate the least squares solution? Where do these values come from? How do the `Initial Values` compare with the result of the least squares solution (the `Object point coordinates`)?**

{ click in this cell and write your answer here }

- **Describe how the least squares method iteratively minimizes the residuals to achieve a solution?**

<div class="alert alert-block alert-info"><b>HINT!</b> Use `print` and look at how the residuals minimize? </div> 

{ click in this cell and write your answer here }

<div class="alert alert-block alert-warning"><b>TASK! </b>  </div>

- **Change the `input` to `./data/inputRes02.txt` and then `./data/inputRes03.txt`. Discuss what you observe. You answer must be between 250 and 300 words.**

{ click in this cell and write your answer here }

_images:_
    
- **Principles of photogrammetry**: EZ.pdh (2023), Aerial Photogrammetry Help, https://ez-pdh.com/aerial-photogrammetry-help/